In [1]:
import numpy as np
import pandas as pd
from keras.layers import Dense, Input, Flatten, concatenate, Conv1D
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D,Bidirectional, Convolution1D, Embedding, SpatialDropout1D, BatchNormalization,MaxPooling1D, Dropout, LSTM, GRU
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints
from keras.models import Model
from keras.layers.merge import Concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [2]:
INPUT_PATH = '../input/'
CACHE_PATH = '../cache/'
OUTPUT_PATH ='../output/'

In [3]:
data = np.load(CACHE_PATH + 'data.npz')
X_train = data['X_train']
y_train = data['y_train']
X_val = data['X_val']
y_val = data['y_val']
X_test = data['X_test']
embedding_matrix = np.load(CACHE_PATH + 'embedding_matrix.npy')

In [4]:
MAX_FEATURES = 20000
MAX_SEQUENCE_LENGTH = 300
EMBEDDING_DIM = 300

In [5]:
embedding_layer = Embedding(MAX_FEATURES,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [6]:
def get_gru_model():
    inp = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(inp)
    x = SpatialDropout1D(0.2)(embedded_sequences)
    x = Bidirectional(GRU(250, dropout=0.25, recurrent_dropout=0.25, return_sequences=True))(embedded_sequences)
    x = Conv1D(64, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool, max_pool])
    x = Dense(100, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(1, activation="linear")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='mse',optimizer='adam')
    return model

In [7]:
def train_gru_model(model):
    model_path = CACHE_PATH + "gru_weights_best.hdf5"
    early = EarlyStopping(monitor="val_loss", mode="min", patience=5)
    checkpoint = ModelCheckpoint(model_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    callbacks_list = [checkpoint, early]
    model.fit(X_train, y_train, batch_size=128, epochs=100, validation_data=(X_val, y_val), callbacks=callbacks_list)
    model.load_weights(model_path)
    return model

In [8]:
model = get_gru_model()

In [9]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 256)     5120000     input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 100, 500)     760500      embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 98, 64)       96064       bidirectional_1[0][0]            
__________________________________________________________________________________________________
global_ave

In [10]:
from keras.utils.vis_utils import plot_model
plot_model(model,to_file='GRU.png')

In [11]:
model = train_gru_model(model)

Train on 258339 samples, validate on 13597 samples
Epoch 1/100
258339/258339 [==============================] - 346s 1ms/step - loss: 0.9110 - val_loss: 0.5956

Epoch 00001: val_loss improved from inf to 0.59556, saving model to ../cache/gru_weights_best.hdf5
Epoch 2/100
258339/258339 [==============================] - 350s 1ms/step - loss: 0.6064 - val_loss: 0.4444

Epoch 00002: val_loss improved from 0.59556 to 0.44443, saving model to ../cache/gru_weights_best.hdf5
Epoch 3/100
258339/258339 [==============================] - 344s 1ms/step - loss: 0.5025 - val_loss: 0.3971

Epoch 00003: val_loss improved from 0.44443 to 0.39707, saving model to ../cache/gru_weights_best.hdf5
Epoch 4/100
258339/258339 [==============================] - 342s 1ms/step - loss: 0.4610 - val_loss: 0.3854

Epoch 00004: val_loss improved from 0.39707 to 0.38540, saving model to ../cache/gru_weights_best.hdf5
Epoch 5/100
258339/258339 [==============================] - 342s 1ms/step - loss: 0.4343 - val_loss:

In [13]:
y_test = model.predict(X_test,batch_size=128,verbose=1)
y_test[y_test < 1] = 1
y_test[y_test > 4.7] = 5

50000/50000 [==============================] - 20s 400us/step


In [15]:
import datetime
time = datetime.datetime.now()

In [17]:
sub = pd.read_csv(INPUT_PATH + 'sample.csv',header=None,names=['Id','Score'])
sub['Score'] = y_test
sub.to_csv(OUTPUT_PATH + 'gru_{}.csv'.format(time.strftime('%Y-%m-%d-%H:%M:%S')),index=False, header=False)